In [55]:
# importing dependencies
import pandas as pd
import numpy as np
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet
from pywsd import disambiguate
from pywsd.similarity import max_similarity as maxsim
from nltk.wsd import lesk
import string

In [56]:
# disambiguate('I went to the bank to deposit my money')
disambiguate('love you', algorithm=maxsim, similarity_option='wup', keepLemmas=True)


[('love', 'love', Synset('love.n.04')), ('you', 'you', None)]

In [57]:
df_csv = pd.read_csv('formspring_all.csv')

In [58]:
df_csv.head()

,q_a,label
0,fuck justin bieber. he's a fucking fag,1
1,its also funny how you stalked my whole twitte...,1
2,hello bitch ! how's ur day going . good i hope...,1
3,hey. why you such a bitch .,1
4,how old are you . and how do you know michael .,1


In [59]:
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(int(df_csv['label'][i]))
    q_a.append(df_csv['q_a'][i])
print(len(q_a))
print(len(label))


12721
12721


In [60]:
# text cleaning
w=list(string.ascii_lowercase)
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("@", '')
    sen=sen.replace("pu**y", 'pussy')
    sen=sen.replace(" mofo ", ' motherfucker')
    sen=sen.replace("NEWLINE_TOKEN", " ")
    sen=sen.replace("TAB_TOKEN", " ")
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("?", ' ? ')
    sen=sen.replace("!", ' ! ')
    sen=sen.replace(" y ", ' why ')
    sen=sen.replace(" u ", ' you ')
    sen=sen.replace(" w ", ' with ')
    sen=sen.replace(" mf ", ' motherfucker ')
    sen=sen.replace(" IDGAF ", ' i dont give a fuck ')
    sen=sen.replace(" IDRC ", ' I Dont Really Care ')
    sen=sen.replace(" tf ", ' the fuck ')
    sen=sen.replace("wtf ", 'what the fuck ')
    sen=sen.replace(" yanno ", ' you know ')
    sen=sen.replace(" igt ", ' i know right ')
    sen=sen.replace(" r ", ' are ')
    sen=sen.replace(" tbh ", ' to be honest ')
    sen=sen.replace(" ik ", ' i know ')
    sen=sen.replace(" af ", ' as fuck ')
    sen=sen.replace(" hes ", ' he is ')
    sen=sen.replace(" des ", ' this ')
    sen=sen.replace(" bout ", ' about ')
    sen=sen.replace(" em ", ' them ')
    sen=sen.replace(" stg ", ' swear to god ')
    sen=sen.replace(" bj ", ' blow job ')
    sen=sen.replace(" ig ", ' i guess ')
    sen=sen.replace(" fagg ", ' faggot ')
    sen=sen.replace(" fag ", ' faggot ')
    sen=sen.replace(" fagot ", ' faggot ')
    sen=sen.replace(" tfw ", ' That Feel When ')
    sen=sen.replace(" wa ", ' was ')
    sen=sen.replace(" n ", ' and ')
    sen=sen.replace(" y0 ", ' you ')
    sen=sen.replace(" dat ", ' that ')
    sen=sen.replace(" yo ", ' you ')
    sen=sen.replace("youre ", ' you are ')
    sen=sen.replace(" v ", ' vagina ')
    sen=sen.replace(" pusy ", ' pussy ')
    sen=sen.replace(" pusy", ' pussy ')
    sen=sen.replace("pussie", 'pussy')
    sen=sen.replace(" v.", ' vagina ')
    sen=sen.replace(" doggystyle ", ' doggy style ')
    sen=sen.replace("di ck", 'dick')
    sen=sen.replace("r you", 'are you')
    sen=sen.replace("r u", 'are you')
    sen=sen.replace("f*ck", 'fuck')
    sen=sen.replace("f**", 'fuck')
    sen=sen.replace(" fxk ", 'fuck')
    sen=sen.replace(" wil ", 'will')
    sen=sen.replace(" il ", 'i will')
    sen=sen.replace(" asss ", ' ass ')
    sen=sen.replace(" lifeles ", ' lifeless ')
    sen=sen.replace(" ugler ", ' uglier ')
    sen=sen.replace(" you're ", ' you are ') 
    sen=sen.replace(",", ' ') 
    sen=sen.replace("?", '.') 
    sen=sen.lower()
    for s in w:
        n=s+s+s
        sen=sen.replace(n, s)
    sen=" ".join(sen.split())
    QA.append(sen)
# print(QA)
q_a=QA
print(len(q_a))

12721


In [61]:
# Read data files
comments_attack3=pd.DataFrame()
comments_attack3['comment']=q_a
comments_attack3['label']=label
dataframe3 = comments_attack3
print(dataframe3['comment'])

0                   fuck justin bieber. he's a fucking fag
1        its also funny how you stalked my whole twitte...
2        hello bitch ! how's ur day going . good i hope...
3                              hey. why you such a bitch .
4          how old are you . and how do you know michael .
                               ...                        
12716    we will wake up to blue skys and keep on going on
12717    take the battery cable off and make her stay t...
12718                       thank you ! ! i am awesome ! !
12719         all the time. sometimes more than once a day
12720       oh o.k ! ! nice make sure it's stolie's please
Name: comment, Length: 12721, dtype: object


In [62]:
# Extention of emaning with all words
def disambiguation(sen):
    text = word_tokenize(sen)
    allWords=nltk.pos_tag(text)
    new_sen=[]
    for x in allWords: 

        try: 
            synonyms = wn.synsets(x[0])
            for syn in synonyms:
                for l in syn.lemmas():
                    ns=sen.replace(x[0], l.name())
                    ns=ns.lower()
                    new_sen.append(ns.replace('_', ' '))
        except:
            ns=sen.lower()
            new_sen.append(ns.replace('_', ' '))
    new_sen = list(dict.fromkeys(new_sen))
    return new_sen

In [63]:
new_q_a=[]
new_label=[]
for i in  range(len(q_a)):   
    collction_sen= disambiguation(q_a[i])
    for x in collction_sen:
        new_q_a.append(x)
        new_label.append(label[i])

In [64]:
#number of extented datas
print(len(new_q_a))
print(len(new_label))

1121831
1121831


In [127]:
# my_dict = { 'q_a':q_a,'label':label}
# df = pd.DataFrame(my_dict)
# df.to_csv('formspring_all',index=False)

In [65]:
# Read data files
comments_attack=pd.DataFrame()
comments_attack['comment']=new_q_a
comments_attack['label']=new_label

In [66]:
dataframe1 = comments_attack

# Text preprocessing
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))



In [67]:
print(dataframe1['label'])

0          1
1          1
2          1
3          1
4          1
          ..
1121826    0
1121827    0
1121828    0
1121829    0
1121830    0
Name: label, Length: 1121831, dtype: int64


In [68]:
print(dataframe1['comment'])

0                     fuck justin bieber. he's a fucking fag
1               fucking justin bieber. he's a fuckinging fag
2                   screw justin bieber. he's a screwing fag
3             screwing justin bieber. he's a screwinging fag
4                       ass justin bieber. he's a assing fag
                                 ...                        
1121826    oh o.k ! ! nice make for certain it's stolie's...
1121827    oh o.k ! ! nice make sure enough it's stolie's...
1121828    oh o.k ! ! nice make sure as shooting it's sto...
1121829    oh o.k ! ! nice make sure information technolo...
1121830      oh o.k ! ! nice make sure it's stolie's delight
Name: comment, Length: 1121831, dtype: object


In [69]:
#split the data into training and validation sets
train_x, valid_x2, train_y, valid_y2 = train_test_split(dataframe1['comment'], dataframe1['label'], test_size=0.001, random_state=42)

In [70]:
df_csv = pd.read_csv('ask_fm_data.csv')

In [71]:
#marge all data from files that has been read

q_a=[]
label=[]
for i in range(len(df_csv)):
    label.append(int(df_csv['label'][i]))
    q_a.append(df_csv['q_a'][i])
print(len(q_a))
print(len(label))
# text cleaning
w=list(string.ascii_lowercase)
QA=[]
for i in q_a:
    sen=i.replace('@USER','')
    sen=sen.replace('@user','')
    sen=sen.replace("#", ' ')
    sen=sen.replace("@", '')
    sen=sen.replace("pu**y", 'pussy')
    sen=sen.replace(" mofo ", ' motherfucker')
    sen=sen.replace("NEWLINE_TOKEN", " ")
    sen=sen.replace("TAB_TOKEN", " ")
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("??", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("? ?", '?')
    sen=sen.replace("?", ' ? ')
    sen=sen.replace("!", ' ! ')
    sen=sen.replace(" y ", ' why ')
    sen=sen.replace(" u ", ' you ')
    sen=sen.replace(" w ", ' with ')
    sen=sen.replace(" mf ", ' motherfucker ')
    sen=sen.replace(" IDGAF ", ' i dont give a fuck ')
    sen=sen.replace(" IDRC ", ' I Dont Really Care ')
    sen=sen.replace(" tf ", ' the fuck ')
    sen=sen.replace("wtf ", 'what the fuck ')
    sen=sen.replace(" yanno ", ' you know ')
    sen=sen.replace(" igt ", ' i know right ')
    sen=sen.replace(" r ", ' are ')
    sen=sen.replace(" tbh ", ' to be honest ')
    sen=sen.replace(" ik ", ' i know ')
    sen=sen.replace(" af ", ' as fuck ')
    sen=sen.replace(" hes ", ' he is ')
    sen=sen.replace(" des ", ' this ')
    sen=sen.replace(" bout ", ' about ')
    sen=sen.replace(" em ", ' them ')
    sen=sen.replace(" stg ", ' swear to god ')
    sen=sen.replace(" bj ", ' blow job ')
    sen=sen.replace(" ig ", ' i guess ')
    sen=sen.replace(" fagg ", ' faggot ')
    sen=sen.replace(" fag ", ' faggot ')
    sen=sen.replace(" fagot ", ' faggot ')
    sen=sen.replace(" tfw ", ' That Feel When ')
    sen=sen.replace(" wa ", ' was ')
    sen=sen.replace(" n ", ' and ')
    sen=sen.replace(" y0 ", ' you ')
    sen=sen.replace(" dat ", ' that ')
    sen=sen.replace(" yo ", ' you ')
    sen=sen.replace("youre ", ' you are ')
    sen=sen.replace(" v ", ' vagina ')
    sen=sen.replace(" pusy ", ' pussy ')
    sen=sen.replace(" pusy", ' pussy ')
    sen=sen.replace("pussie", 'pussy')
    sen=sen.replace(" v.", ' vagina ')
    sen=sen.replace(" doggystyle ", ' doggy style ')
    sen=sen.replace("di ck", 'dick')
    sen=sen.replace("r you", 'are you')
    sen=sen.replace("r u", 'are you')
    sen=sen.replace("f*ck", 'fuck')
    sen=sen.replace("f**", 'fuck')
    sen=sen.replace(" fxk ", 'fuck')
    sen=sen.replace(" wil ", 'will')
    sen=sen.replace(" il ", 'i will')
    sen=sen.replace(" asss ", ' ass ')
    sen=sen.replace(" lifeles ", ' lifeless ')
    sen=sen.replace(" ugler ", ' uglier ')
    sen=sen.replace(" you're ", ' you are ') 
    sen=sen.replace(",", ' ') 
    sen=sen.replace("?", '.') 
    sen=sen.lower()
    for s in w:
        n=s+s+s
        sen=sen.replace(n, s)
    sen=" ".join(sen.split())
    QA.append(sen)
# print(QA)
q_a=QA
print(len(q_a))

9998
9998
9998


In [72]:
# Read data files
comments_attack2=pd.DataFrame()
comments_attack2['comment2']=q_a
comments_attack2['label2']=label

In [73]:
dataframe = comments_attack2

# Text preprocessing
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
# dataframe['comment'] = dataframe['comment'].apply(lambda x: x.lower())
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[/(){}\[\]\|@,;]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub('[^0-9a-z #+_]','',x)))
# dataframe['comment'] = dataframe['comment'].apply((lambda x: re.sub(' +',' ',x)))


In [74]:
#split the data into training and validation sets
train_x2, valid_x, train_2, valid_y = train_test_split(dataframe['comment2'], dataframe['label2'], test_size=0.9, random_state=42)

In [75]:
print('train comments length: ',len(train_x))
print('test comments length: ',len(valid_x))

print('train comments length: ',len(train_y))
print('test comments length: ',len(valid_y))

train comments length:  1120709
test comments length:  8999
train comments length:  1120709
test comments length:  8999


In [76]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('data/wiki-news-300d-1M.vec', encoding="utf8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(dataframe3['comment'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure e
# qual length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [77]:
# convert to one_hot
train_y_onehot = keras.utils.to_categorical(train_y, 3)
valid_y_onehot = keras.utils.to_categorical(valid_y, 3)

In [78]:
def cnn(xtrain, ytrain, xvalid, yvalid, epochs = 1):
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(3, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer='adam', loss='categorical_crossentropy',  metrics=['accuracy'])
    model.fit(xtrain, ytrain,
              batch_size=256,
              epochs=epochs)
    predictions = model.predict(xvalid)
    predictions = predictions.argmax(axis=-1)
    accuracy = model.evaluate(xvalid, yvalid, verbose=0)
    f1score = metrics.f1_score(valid_y, predictions, average='weighted')
    return accuracy, f1score

In [ ]:
#askfm 100k tasining datasets and fromspring 11k as a test datasets
accuracy, f1score = cnn(train_seq_x, train_y_onehot, valid_seq_x, valid_y_onehot)
print("CNN, Word Embeddings acuuracy accuracy:%s     f1 score: %s"% (accuracy[1], f1score))

Epoch 1/1
 782080/1120709 [===================>..........] - ETA: 6:49 - loss: 0.0502 - accuracy: 0.9836